In [1]:
from threading import Thread, Lock

lock = Lock()
def Print(*args):
    with lock:
        print(*args)

Print("hi")

hi


In [2]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer

# Admin

In [3]:
broker = "localhost:9092"
admin = KafkaAdminClient(bootstrap_servers=[broker])

In [4]:
admin.list_topics()

[]

In [21]:
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError

In [15]:
# admin.delete_topics(["even_nums", "odd_nums"])

In [22]:
try:
    admin.create_topics([NewTopic("even_nums", num_partitions=1, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

already exists


In [23]:
try:
    admin.create_topics([NewTopic("odd_nums", num_partitions=2, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

already exists


In [24]:
admin.list_topics()

['even_nums', 'odd_nums']

# Producer

In [25]:
producer = KafkaProducer(bootstrap_servers=[broker])

In [27]:
result = producer.send("even_nums", bytes(str(0), "utf-8"))

In [29]:
result.get()

RecordMetadata(topic='even_nums', partition=0, topic_partition=TopicPartition(topic='even_nums', partition=0), offset=0, timestamp=1700066230422, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=1, serialized_header_size=-1)

In [32]:
import time, threading

def num_producer(topic, start, step):
    producer = KafkaProducer(bootstrap_servers=[broker])
    num = start
    while True:
        if num < 10:
            Print("send", num, "to", topic)
        producer.send(topic, bytes(str(num), "utf-8"))
        num += step
        time.sleep(1)

# TODO: have two threads for this function
threading.Thread(target=num_producer, args=("even_nums", 0, 2)).start()
threading.Thread(target=num_producer, args=("odd_nums", 1, 2)).start()

send 1 to odd_nums
send 0 to even_nums
send 3 to odd_nums
send 2 to even_nums
send 5 to odd_nums
send 4 to even_nums
send 7 to odd_nums
send 6 to even_nums
send 9 to odd_nums
send 8 to even_nums


# Consumer

In [33]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
batch = consumer.poll(1000)
batch

{}

In [34]:
consumer.assignment()

set()

## Manual Assignment

In [36]:
from kafka import TopicPartition

In [37]:
consumer.assign([TopicPartition("even_nums", 0)])
consumer.assignment()

{TopicPartition(topic='even_nums', partition=0)}

In [48]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

492
494
496
498


## Automatic Assignment